In [136]:
import yfinance as yf
import pandas as pd
import datetime as dt
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [137]:
instruments = ["FB"]
index = ["SPY"]
start = "2014-01-01"
end = dt.date.today()

In [138]:
if len(instruments) != 0:
    choice = ""
    
    for ele in instruments:  
        choice += ele + " "
        
else: 
    choice = index

print(choice)
type(choice)

FB 


str

In [139]:
def get_data(choice, start, end):
    data = yf.download(choice, start=start, end=end, group_by='tickers')
    return data

In [140]:
data = get_data(choice, start, end)
data = data.drop(columns=["Adj Close"])

[*********************100%***********************]  1 of 1 completed


In [141]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2013-12-31,54.119999,54.860001,53.910000,54.650002,43076200
2014-01-02,54.830002,55.220001,54.189999,54.709999,43195500
2014-01-03,55.020000,55.650002,54.529999,54.560001,38246200
2014-01-06,54.419998,57.259998,54.049999,57.200001,68852600
2014-01-07,57.700001,58.549999,57.220001,57.919998,77207400
...,...,...,...,...,...
2020-09-15,270.670013,274.519989,269.299988,272.420013,18478500
2020-09-16,267.290009,272.440002,261.790009,263.519989,29183400
2020-09-17,258.279999,261.500000,250.190002,254.820007,31281400


In [142]:
data = momentum_indicator(data, period=4)
data = sy_indicator(data)
data = asy5_indicator(data)
data = Moving_average(data, window=5)

In [143]:
data

,Open,High,Low,Close,Volume,Momentum_4,Sy,Asy5,MA5
Date,,,,,,,,,
2013-12-31,54.119999,54.860001,53.910000,54.650002,43076200,NaN,NaN,NaN,NaN
2014-01-02,54.830002,55.220001,54.189999,54.709999,43195500,NaN,0.109725,NaN,NaN
2014-01-03,55.020000,55.650002,54.529999,54.560001,38246200,NaN,-0.274545,NaN,NaN
2014-01-06,54.419998,57.259998,54.049999,57.200001,68852600,NaN,4.725287,NaN,NaN
2014-01-07,57.700001,58.549999,57.220001,57.919998,77207400,3.269997,1.250880,NaN,55.808000
...,...,...,...,...,...,...,...,...,...
2020-09-15,270.670013,274.519989,269.299988,272.420013,18478500,-1.299988,2.328501,0.092720,269.397998
2020-09-16,267.290009,272.440002,261.790009,263.519989,29183400,-4.570007,-3.321582,-0.759529,267.357996
2020-09-17,258.279999,261.500000,250.190002,254.820007,31281400,-11.789978,-3.357178,-1.015306,264.703998


# Regression Label

In [144]:
def create_label(data, timeframe=-1):
    data['Label'] = data["Close"].shift(timeframe)
    return data

In [145]:
data = create_label(data, -1)

In [146]:
data.head(10)

,Open,High,Low,Close,Volume,Momentum_4,Sy,Asy5,MA5,Label
Date,,,,,,,,,,
2013-12-31,54.119999,54.860001,53.910000,54.650002,43076200,NaN,NaN,NaN,NaN,54.709999
2014-01-02,54.830002,55.220001,54.189999,54.709999,43195500,NaN,0.109725,NaN,NaN,54.560001
2014-01-03,55.020000,55.650002,54.529999,54.560001,38246200,NaN,-0.274545,NaN,NaN,57.200001
2014-01-06,54.419998,57.259998,54.049999,57.200001,68852600,NaN,4.725287,NaN,NaN,57.919998
2014-01-07,57.700001,58.549999,57.220001,57.919998,77207400,3.269997,1.250880,NaN,55.808000,58.230000
2014-01-08,57.599998,58.410000,57.230000,58.230000,56682400,3.520000,0.533796,1.269029,56.524000,57.220001
2014-01-09,58.650002,58.959999,56.650002,57.220001,92253300,2.660000,-1.749717,0.897140,57.026000,57.939999
2014-01-10,57.130001,58.299999,57.060001,57.939999,42449500,0.739998,1.250446,1.202139,57.702000,55.910000
2014-01-13,57.910000,58.250000,55.380001,55.910000,63010900,-2.009998,-3.566472,-0.456213,57.443999,57.740002


# Drop NaN Values

In [147]:
data = data.dropna()

In [148]:
data

,Open,High,Low,Close,Volume,Momentum_4,Sy,Asy5,MA5,Label
Date,,,,,,,,,,
2014-01-08,57.599998,58.410000,57.230000,58.230000,56682400,3.520000,0.533796,1.269029,56.524000,57.220001
2014-01-09,58.650002,58.959999,56.650002,57.220001,92253300,2.660000,-1.749717,0.897140,57.026000,57.939999
2014-01-10,57.130001,58.299999,57.060001,57.939999,42449500,0.739998,1.250446,1.202139,57.702000,55.910000
2014-01-13,57.910000,58.250000,55.380001,55.910000,63010900,-2.009998,-3.566472,-0.456213,57.443999,57.740002
2014-01-14,56.459999,57.779999,56.099998,57.740002,37503600,-0.489998,3.220695,-0.062250,57.408000,57.599998
...,...,...,...,...,...,...,...,...,...,...
2020-09-14,270.950012,276.640015,265.700012,266.149994,24093800,-5.010010,-0.172682,-1.208648,269.145996,272.420013
2020-09-15,270.670013,274.519989,269.299988,272.420013,18478500,-1.299988,2.328501,0.092720,269.397998,263.519989
2020-09-16,267.290009,272.440002,261.790009,263.519989,29183400,-4.570007,-3.321582,-0.759529,267.357996,254.820007


# Split Data

In [149]:
def split_data(percentage, df, dropLabels):
    splitValue = int(len(df) * percentage) # Used for calculating the split for test train based on data size
    
    x = np.array(df.drop(dropLabels, 1)) # Drop all unused columns
    y = np.ravel(df.Label) # take only labels

    x_train, x_test = x[:splitValue], x[splitValue:]
    y_train, y_test = y[:splitValue], y[splitValue:]
    
    return x_train, x_test, y_train, y_test


x_train, x_test, y_train, y_test = split_data(.8, data, ['Label', 'Open', 'High', 'Low' , 'Volume', 'Momentum_4','Sy', "Asy5", "MA5"])

In [150]:
x_train

array([[ 58.22999954],
       [ 57.22000122],
       [ 57.93999863],
       ...,
       [186.27000427],
       [186.99000549],
       [185.30000305]])

# MinMax Scaler

In [151]:
def MM_Scaler(x_train, x_test):

    linearScaler = MinMaxScaler().fit(x_train)

    # Scale train and test
    x_train = linearScaler.transform(x_train)
    x_test = linearScaler.transform(x_test)
    return x_train, x_test

In [152]:
x_train, x_test = MM_Scaler(x_train, x_test)

In [153]:
x_train

array([[0.02866378],
       [0.02250413],
       [0.02689516],
       ...,
       [0.80953836],
       [0.81392941],
       [0.80362263]])

# Type 1

In [25]:
def momentum_indicator(df, period=4):
    momentum = pd.Series(df['Close'].diff(period), name='Momentum_' + str(period))
    df = df.join(momentum)
    return df

In [ ]:
#def rate_of_change(df, period=4):
#    df_daily['Price_ROC'] = df_daily.Close.rolling(window=2).apply(lambda x: ((x.iloc[1] / x.iloc[0]) - 1) * 100)

# Type 2

In [ ]:
#df_daily['BIAS6'] = df_daily.Close.rolling(window=6).apply(lambda x: ((x.iloc[5] - ((x.iloc[0] + x.iloc[1] + x.iloc[2] + x.iloc[3] + x.iloc[4] + x.iloc[5]) / 6)) / (x.iloc[0] + x.iloc[1] + x.iloc[2] + x.iloc[3] + x.iloc[4] + x.iloc[5] / 6) * 100))

In [52]:
def sy_indicator(df):
    sy = pd.Series(np.log(df['Close']) - np.log(df['Close'].shift(1)) , name='Sy')
    sy = sy * 100
    df = df.join(sy)
    return df

In [53]:
def asy5_indicator(df):
    df['Asy5'] = df["Sy"].rolling(window=5).sum()/5
    return df

In [71]:
def Moving_average(df, window=5):
    df['MA'+ str(window)] = df["Close"].rolling(window=window).sum()/window
    return df